In [72]:
from numpy import *
import numpy as np
from numpy.linalg import eig, norm

from prettytable import PrettyTable

In [73]:
def power_method(A, num_iter, x_initial):
    """
    Computes the power method for the dominant eigenvalue

    :param A: square input matrix
    :param num_iter: number of iterations
    :param x_initial: initial guess for the eigenvector (n x 1)
    :return: estimated_eigenvalues_vector, estimated_eigenvector_matrix
    - estimated_eigenvalues_vector
    the sequence of estimated eigenvalue approximations n x 1 vector
    - estimated_eigenvector_matrix
    the sequence of corresponding eigenvector approximations (along columns) - INCLUDING initial guess as the first
    iteration
    """

    # square matrix, so both dimensions should match
    n, m = A.shape
    assert n == m

    # include initial condition as well
    new_length = num_iter + 1

    # get accumulators for both the eigenvalues and eigenvectors
    estimated_eigenvalues_vector = np.zeros((new_length, ))
    estimated_eigenvector_matrix = np.zeros((n, new_length))

    # normalize using 2-norm
    x = x_initial / norm(x_initial)

    # get initial estimates for the eigenvalue and eigenvector (just initial guess)
    estimated_eigenvalues_vector[0] = x.T @ A @ x
    estimated_eigenvector_matrix[:, 0] = x

    # start at the 2nd iteration (i = 1, after the initial guess) up to and including i = num_iter
    for i in range(1, new_length):
        z = A @ x
        x = z / norm(z)

        estimated_eigenvalues_vector[i] = x.T @ A @ x
        estimated_eigenvector_matrix[:, i] = x

    return estimated_eigenvalues_vector, estimated_eigenvector_matrix


In [74]:
# Problem 3

# get largest eigenvalue, eigenvector from built-in
A = np.array([[3, 1, 4, 1],
              [5, 9, 2, 6],
              [5, 3, 5, 8],
              [9, 7, 9, 3]])

x_initial = np.array([1/2, 1/2, 1/2, 1/2]).T

# get vector of eigenvalues, and matrix of eigenvectors along the columns corresponding to eigenvalues
eigenvalues, eigenvectors = eig(A)

print(f"Eigenvalues:")
print(eigenvalues)

print(f"Eigenvectors:")
print(eigenvectors)

# need to find the largest eigenvalue, eigenvector (NOT sorted above)
print(f"Arg sort")
argsort_idx = np.argsort(eigenvalues)
print(argsort_idx)

print(f"Index corresponding to largest eigenvalue/eigenvector")
largest_idx = argsort_idx[-1]
print(largest_idx)
second_largest_idx = argsort_idx[-2]
print(second_largest_idx)

print(f"Largest eigenvalues/eigenvectors")
largest_eigenvalue = eigenvalues[largest_idx]
largest_eigenvector = eigenvectors[:, largest_idx]

print(f"lambda_1 = {largest_eigenvalue}")
print(f"x_1 = {largest_eigenvector}")

print(f"Second largest eigenvalues/eigenvectors")
second_largest_eigenvalue = eigenvalues[second_largest_idx]
second_largest_eigenvector = eigenvectors[:, second_largest_idx]

print(f"lambda_2 = {second_largest_eigenvalue}")
print(f"x_2 = {second_largest_eigenvector}")


Eigenvalues:
[19.5493636351772331 -4.6514328443994710 -0.2031439340646978
 5.3052131432869238]
Eigenvectors:
[[-0.1971481203133796 0.2690132302412906 -0.7834070178427704
  0.3291747999755490]
 [-0.5470046454879840 -0.3086522848627238 0.2909560620323353
  -0.8582078250597805]
 [-0.5231462459818992 -0.6075720941531080 0.5486878035729581
  0.3716879277241869]
 [-0.6230863036422100 0.6806039960289487 0.0236581607826158
  0.1302741895055122]]
Arg sort
[1 2 3 0]
Index corresponding to largest eigenvalue/eigenvector
0
3
Largest eigenvalues/eigenvectors
lambda_1 = 19.549363635177233
x_1 = [-0.1971481203133796 -0.5470046454879840 -0.5231462459818992
 -0.6230863036422100]
Second largest eigenvalues/eigenvectors
lambda_2 = 5.305213143286924
x_2 = [0.3291747999755490 -0.8582078250597805 0.3716879277241869
 0.1302741895055122]


In [75]:
# compute power method
num_iter = 20
# the new length from i = 0 to num_iter (total length is num_iter + 1)
new_length = num_iter + 1
estimated_eigenvalues_vector, estimated_eigenvector_matrix = power_method(A, num_iter, x_initial)

# compute errors of eigenvalue
error_eigenvalue_vector = np.fabs(largest_eigenvalue - estimated_eigenvalues_vector)

# compute factor in each error decreases (eigenvalue)
error_factor_eigenvalue_vector = np.zeros((new_length,))
error_factor_eigenvalue_vector[0] = None
error_factor_eigenvalue_vector[1:] = error_eigenvalue_vector[1:] / error_eigenvalue_vector[:-1]

# compute errors of eigenvector (2-norm)
errors_eigenvector_vector = np.zeros((new_length,))

# since the largest eigenvalue for A is positive in this case, we don't have to worry about alternating signs
# Edit: actually, the approximated eigenvector has the opposite sign so add a negative sign below
for i in range(new_length):
    errors_eigenvector_vector[i] = norm(largest_eigenvector - (-estimated_eigenvector_matrix[:, i]))

# compute factor in each error decreases (eigenvector)
error_factor_eigenvector_vector = np.zeros((new_length,))
error_factor_eigenvector_vector[0] = None
error_factor_eigenvector_vector[1:] = errors_eigenvector_vector[1:] / errors_eigenvector_vector[:-1]

# print table out
table = PrettyTable()
table.field_names = ["Iteration number (i)",
                     "Estimated Eigenvalue", "Error (eigenvalue)", "Factor error (eigenvalue)",
                     "Error (eigenvector)", "Factor error (eigenvector)"]

table.add_rows(
    [
        [i,
         f"{estimated_eigenvalues_vector[i]: e}", f"{error_eigenvalue_vector[i]: e}",
         f"{error_factor_eigenvalue_vector[i]: e}",
         f"{errors_eigenvector_vector[i]: e}", f"{error_factor_eigenvector_vector[i]: e}"]
        for i in range(new_length)
    ]
)

print(table)


+----------------------+----------------------+--------------------+---------------------------+---------------------+----------------------------+
| Iteration number (i) | Estimated Eigenvalue | Error (eigenvalue) | Factor error (eigenvalue) | Error (eigenvector) | Factor error (eigenvector) |
+----------------------+----------------------+--------------------+---------------------------+---------------------+----------------------------+
|          0           |     2.000000e+01     |    4.506364e-01    |             nan           |     3.310811e-01    |             nan            |
|          1           |     1.948547e+01     |    6.388877e-02    |        1.417746e-01       |     5.647455e-02    |        1.705762e-01        |
|          2           |     1.954898e+01     |    3.840015e-04    |        6.010469e-03       |     1.276838e-02    |        2.260909e-01        |
|          3           |     1.955153e+01     |    2.161631e-03    |        5.629226e+00       |     3.435705e-0

In [76]:
# compare convergence with |lambda_2 / lambda_1|
convergence_factor = math.fabs(second_largest_eigenvalue / largest_eigenvalue)

print("Expected convergence factor")
print(f"{convergence_factor: e}")


Expected convergence factor
 2.713752e-01


In [77]:
# print out stats for comparison above

# format to print 16 digits of precision
float_formatter = "{:.16f}".format
np.set_printoptions(formatter={'float_kind': float_formatter})

for i in range(new_length):
    print(estimated_eigenvector_matrix[:, i])

print(errors_eigenvector_vector)


[0.5000000000000000 0.5000000000000000 0.5000000000000000
 0.5000000000000000]
[0.2127237566611795 0.5199914051717720 0.4963554322094187
 0.6618072429458917]
[0.1948413042184071 0.5482180795710460 0.5324855519012367
 0.6147787735587006]
[0.1983212093742582 0.5453214277737873 0.5216763897023247
 0.6254174467907319]
[0.1970371280543225 0.5470298749398500 0.5237037789935753
 0.6226307396517178]
[0.1972202406467097 0.5468970185319886 0.5230701786377207
 0.6232218032289777]
[0.1971433687742005 0.5470026845153875 0.5231797096386678
 0.6230614309229215]
[0.1971526183598535 0.5469976313052146 0.5231424542097581
 0.6230942216260152]
[0.1971479639486950 0.5470042843290208 0.5231482799172709
 0.6230849624699514]
[0.1971484055193970 0.5470041805136605 0.5231460691733097
 0.6230867700491263]
[0.1971481197551136 0.5470046066184856 0.5231463713983130
 0.6230862326419654]
[0.1971481387101914 0.5470046141651536 0.5231462387597904
 0.6230863313832387]
[0.1971481208925690 0.5470046419307032 0.52314625383

In [78]:
# FIXME: debug stuff
# [0.3310810845918672 0.0564745468611432 0.0127683794552538
#  0.0034357045390493 0.0007289289508271 0.0002023148116206
#  0.0000420106061278 0.0000121038500286 0.0000024678982626
#  0.0000007391483496 0.0000001492697597 0.0000000462740307
#  0.0000000093853895 0.0000000029782495 0.0000000006162937
#  0.0000000001971235 0.0000000000421380 0.0000000000133865
#  0.0000000000029734 0.0000000000009289 0.0000000000002143
#  0.0000000000000655 0.0000000000000156 0.0000000000000048
#  0.0000000000000011 0.0000000000000000]

# [0.1971481203134360 0.5470046454878352 0.5231462459820425
#  0.6230863036422025]
# [0.1971481203134057 0.5470046454879316 0.5231462459819139
#  0.6230863036422354]
# [0.1971481203133841 0.5470046454879727 0.5231462459819090
#  0.6230863036422105]
# [0.1971481203133814 0.5470046454879802 0.5231462459819005
#  0.6230863036422118]
# [0.1971481203133799 0.5470046454879831 0.5231462459818997
#  0.6230863036422102]
# [0.1971481203133797 0.5470046454879838 0.5231462459818993
#  0.6230863036422102]

# [0.3310810845918672 0.0564745468611432 0.0127683794552538
#  0.0034357045390493 0.0007289289508271 0.0002023148116206
#  0.0000420106061278 0.0000121038500286 0.0000024678982626
#  0.0000007391483496 0.0000001492697597 0.0000000462740307
#  0.0000000093853895 0.0000000029782495 0.0000000006162937
#  0.0000000001971235 0.0000000000421380 0.0000000000133865
#  0.0000000000029734 0.0000000000009289 0.0000000000000000]

# [0.1971481203134360 0.5470046454878352 0.5231462459820425
#  0.6230863036422025]

print(norm(largest_eigenvector - (-estimated_eigenvector_matrix[:, num_iter])))


2.1428576348218256e-13
